In [10]:
import pickle
import numpy as np
import datetime
import nltk

In [3]:
df = pickle.load(open('/mnt/mypartition/pubmed_nlp_research/C_article_replication_and_translational_classification/pickled_objects/dfJournals_trans_categories.p', 'rb'))

In [6]:
#check labels- great
print df.columns.values
print df['sentiment'].unique()

df.columns = ['label', 'title', 'abstract', 'qualifier_terms', 'descriptor_terms']

['sentiment' 'title' 'abstract' 'qualifier_terms' 'descriptor_terms']
['T0' 'T1/T2' 'T3/T4']


In [11]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [7]:
df[0:10]

,label,title,abstract,qualifier_terms,descriptor_terms
0,T0,Quantification of the relative contributions o...,Amyotrophic lateral sclerosis (ALS) and fronto...,,
1,T0,Ubiquitin Specific Protease 36 (USP36) control...,Ubiquitination of the TrkA neurotrophin recept...,,
2,T0,Functional Diversity of Cytotoxic tRNase/immun...,Contact-dependent growth inhibition (CDI) is a...,,
3,T0,Identification of a distinct substrate binding...,The type III secretion system (T3SS) effector ...,,
4,T0,Role of chondroitin sulfate (CS) modification ...,Protein tyrosine phosphatase receptor type Z (...,,
5,T0,Chemically programmed bispecific antibodies in...,Chemically programmed bispecific antibodies (b...,,
6,T0,Non-mutagenic Suppression of Enterocyte Ferrop...,Iron transfer across the basolateral membrane ...,,
7,T0,Structural plasticity of cholesteryl ester tra...,Cholesteryl ester transfer protein (CETP) medi...,,
8,T0,Identification of a Membrane-Bound Prepore Spe...,Pore-forming toxins (PFT) are cytolytic protei...,,
9,T0,EspR-dependent ESAT-6 secretion of Mycobacteri...,Attenuation of M. bovis BCG strain is related ...,,


### Generate probssentences 

In [12]:
#add boolean labels
df['boolean_label'] = 1
for row in df.iterrows():
    if row[1]['label'] != 'T0':
        df.set_value(row[0], 'boolean_label', 0)

In [16]:
#not doing stratified but just shuffling to match method used for probssentences (movie reviews)
#do a larger split
fracTrain = 0.80
nSamples = df.shape[0]
order = np.random.permutation(nSamples) # come up with a random ordering
splitIndex = int(np.round(nSamples*fracTrain))
train1 = df.ix[order[:splitIndex],:]
test1 = df.ix[order[splitIndex:],:]

In [17]:
print train1.shape
print test1.shape

(77639, 6)
(19410, 6)


In [18]:
train1[0:10]

,label,title,abstract,qualifier_terms,descriptor_terms,boolean_label
83452,T3/T4,Validity and Reliability of the US National Ca...,To integrate the patient perspective into adve...,classification adverse effects adverse effects...,Adult Adverse Drug Reaction Reporting Systems ...,0
29602,T0,Monospecific inhibitors show that both mannan-...,The lectin pathway is an antibody-independent ...,antagonists & inhibitors chemistry metabolism ...,"Complement Pathway, Mannose-Binding Lectin Cry...",1
13691,T0,NdhP is an exclusive subunit of large complex ...,Two major complexes of NADPH dehydrogenase (ND...,chemistry metabolism chemistry deficiency gene...,Amino Acid Sequence Cell Respiration Electron ...,1
49017,T1/T2,Phase I pharmacokinetic study of S-1 granules ...,We performed a pharmacokinetic phase I trial o...,administration & dosage pharmacokinetics thera...,Aged Antineoplastic Combined Chemotherapy Prot...,0
67554,T3/T4,Increasing medical team cohesion and leadershi...,Current national health care issues of afforda...,standards standards organization & administration,Adult Clinical Competence Family Practice Feed...,0
69119,T3/T4,[New approach in the surgical treatment of mit...,Severe mitral regurgitation due to prolapse of...,methods pathology physiopathology pathology ph...,"Clinical Trials as Topic Echocardiography, Tra...",0
20993,T0,STAT5 outcompetes STAT3 to regulate the expres...,Inappropriate activation of the transcription ...,metabolism genetics metabolism genetics metabo...,"Acetylation Breast Breast Neoplasms Cell Line,...",1
67148,T3/T4,Triangulating data to improve care: case study.,The implementation of neonatal nursing indicat...,methods methods organization & administration ...,Benchmarking England Health Plan Implementatio...,0
6725,T0,Interplay between TAp73 Protein and Selected A...,"Unlike p53, which is mutated at a high rate in...",genetics biosynthesis genetics genetics metabo...,"Cell Line, Tumor Cell Proliferation Cyclin D1 ...",1
33652,T0,Cell adhesion receptor GPR133 couples to Gs pr...,"Adhesion G protein-coupled receptors (GPCR), w...",metabolism metabolism metabolism metabolism,Animals COS Cells Cell Adhesion Cell Adhesion ...,1


In [19]:
"""
docprob takes two lists
* docs: a list of documents, each of which is a list of sentences
* models: the candidate word2vec models (each potential class)

it returns the array of class probabilities.  Everything is done in-memory.
"""

def docprobsentences(docs, mods):
    # score() takes a list [s] of sentences here; could also be a sentence generator
    sentlist = [s for d in docs for s in d]
    # the log likelihood of each sentence in this review under each w2v representation
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    # now exponentiate to get likelihoods, 
    lhd = np.exp(llhd - llhd.max(axis=0)) # subtract row max to avoid numeric overload
    # normalize across models (stars) to get sentence-star probabilities
    #all this transposing business does is make it so the total probability of a word 
    #   equals 1 between the 2 arrays (positive prob and negative).
    #and the pandas data frame just puts everything into rows/columns format for easy viz
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    # and finally average the sentence probabilities to get the review probability
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    return prob

In [23]:
a = datetime.datetime.now().replace(microsecond=0)

# ****** Split the labeled and unlabeled training sets into clean sentences
#
sentences_pos = []  # Initialize an empty list of sentences
sentences_neg = []  # Initialize an empty list of sentences

inxs_pos = np.where(train1["boolean_label"] == 1)[0].tolist()
inxs_neg = np.where(train1["boolean_label"] == 0)[0].tolist()

print "Parsing sentences from training set"
for inx in inxs_pos:
    review = train1["abstract"].iloc[inx]
    sentences_pos += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

for inx in inxs_neg:
    review = train1["abstract"].iloc[inx]
    sentences_neg += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)    


# ****** Split the labeled and unlabeled training sets into clean sentences
#
sentences = []  # Initialize an empty list of sentences
docsTrain = []
for review in train1["abstract"]:
    result = KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
    sentences += result
    docsTrain.append(result)
        
        
print "Building Word2Vec model"
from gensim.models import Word2Vec
import multiprocessing

## create a w2v learner 
basemodel = Word2Vec(
    workers=multiprocessing.cpu_count(), # use your cores
    iter=3, # iter = sweeps of SGD through the data; more is better
    hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
    )       
basemodel.build_vocab(sentences) 
        
from copy import deepcopy
models = [deepcopy(basemodel) for i in range(2)]
models[0].train(sentences_neg, total_examples=len(sentences_neg) )
models[1].train(sentences_pos, total_examples=len(sentences_pos) )

print "Parsing sentences from test set"
# read in the test set as a list of a list of words
docs = []
for review in test1["abstract"]:
    docs.append(KaggleWord2VecUtility.review_to_sentences(review, tokenizer))

print "scoring test set"
# get the probs (note we give docprob our test set plus the models)
#probs = docprob(docs,models).astype(object)
probssentencesArticle = docprobsentences(docs,models).astype(object)

probssentences_train = docprobsentences(docsTrain,models).astype(object)

pickle.dump(probssentencesArticle, open('./pickled_objects/probssentencesJournalsDataset.p', 'wb'))
pickle.dump(probssentences_train, open('./pickled_objects/probssentencesJournalsDataset_train.p', 'wb'))
        
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

Parsing sentences from training set
Building Word2Vec model
Parsing sentences from test set


/home/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.controlled-trials.com/isrctn/pf/24165302." looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/home/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.fp7-improved.eu/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)


scoring test set
0:08:54


In [21]:
print probssentencesArticle.shape

(1541, 3)


In [30]:
#print probssentencesArticle
print len(probssentencesArticle['doc'].unique())

177


In [29]:
pickle.dump(probssentencesArticle, open('./pickled_objects/probssentencesJournalsDataset.p', 'wb'))

### Do the same for probssentences_train

In [31]:
# read in the training set as a list of a list of words
docs_train = []
for review in train1["abstract"]:
    docs_train.append(KaggleWord2VecUtility.review_to_sentences(review, tokenizer))

In [36]:
probssentences_train_Article = docprobsentences(docs_train,models).astype(object)

In [37]:
print probssentences_train_Article.shape

(1507, 3)


In [40]:
print len(probssentences_train_Article['doc'].unique())

177


In [42]:
pickle.dump(probssentences_train_Article, open('./pickled_objects/probssentences_train_Article.p', 'wb'))

In [43]:
pickle.dump(test1, open('./pickled_objects/articleTest1.p', 'wb'))

In [44]:
pickle.dump(train1, open('./pickled_objects/articleTrain1.p', 'wb'))

In [45]:
print train1.shape
print test1.shape

(177, 7)
(177, 7)


In [47]:
train1[0:10]

,pmid,title,abstract,qualifier_terms,descriptor_terms,label,boolean_label
324,24556849,Emergence of mild cognitive impairment in late...,It is difficult to reliably detect the earlies...,prevention & control diagnosis etiology psycho...,Age Factors Alzheimer Disease Cognition Compar...,T3/T4,0
272,23688215,Recognition of famous names predicts cognitive...,The ability to recognize familiar people is im...,diagnosis physiopathology psychology physiopat...,"Aged Aged, 80 and over Alzheimer Disease Brain...",T0,1
240,21702704,Examining the aging process through the stress...,The aging process is marked by a series of tra...,psychology psychology psychology physiopathology,"Adaptation, Psychological Aged Aging Automobil...",TX,0
169,20461380,"A first in human study of SB-743921, a kinesin...","To determine the maximum-tolerated dose (MTD),...",pharmacology therapeutic use pharmacology ther...,"Adult Aged Aged, 80 and over Antineoplastic Ag...",T1/T2,0
158,21318565,A prospective open-label study of aripiprazole...,Fragile X syndrome (FXS) is the most common in...,drug effects administration & dosage adverse e...,Adolescent Adult Aggression Antipsychotic Agen...,T1/T2,0
97,21106989,"CP-690,550, a therapeutic agent, inhibits cyto...","The retrovirus, human T-cell-lymphotrophic vir...",drug effects metabolism pharmacology drug effe...,"Adult Animals Cell Line, Tumor Cell Proliferat...",T0,1
210,22686210,The relevancy of community-based methods: usin...,"The rates of obesity, diabetes, and heart dise...",epidemiology methods statistics & numerical da...,Adult Alaska Community-Based Participatory Res...,T3/T4,0
121,24623140,Cytosolic phospholipase A2 protein as a novel ...,The objective of this study was to investigate...,administration & dosage methods genetics admin...,Animals Butadienes Drug Delivery Systems Enzym...,T0,1
179,23756294,Utilization of a benchmarking database to info...,An integral part of NICHE (Nurses Improving Ca...,psychology,Aged Attitude of Health Personnel Benchmarking...,T3/T4,0
329,22613170,Frontal gray matter reduction after breast can...,Cognitive changes related to cancer and its tr...,adverse effects therapeutic use pathology drug...,Adult Aged Antineoplastic Combined Chemotherap...,T3/T4,0


### Kaggle Word2VecUtility

In [21]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords


class KaggleWord2VecUtility(object):
    """KaggleWord2VecUtility is a utility class for processing raw HTML text into segments for further learning"""

    @staticmethod
    def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        #
        # 5. Return a list of words
        return(words)
    
    @staticmethod
    def review_to_words( review, remove_stopwords=False ):
        # Function to convert a raw review to a string of words
        # The input is a single string (a raw movie review), and 
        # the output is a single string (a preprocessed movie review)
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text() 
        #
        # 2. Remove non-letters        
        review_text = re.sub("[^a-zA-Z]", " ", review_text) 
        #
        # 3. Convert to lower case, split into individual words
        words = review_text.lower().split()                             
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]   
        #
        # 6. Join the words back into one string separated by space, 
        # and return the result.
        return( " ".join( words ))   

    # Define a function to split a review into parsed sentences
    @staticmethod
    def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append( KaggleWord2VecUtility.review_to_wordlist( raw_sentence, \
                  remove_stopwords ))
        #
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences